# Writing Formatted Output

## Overview

### Questions

* How do I display status information during a simulation?
* How can I log user-defined quantities?
* How can I write formatted output to a text file?

### Objectives

* Demonstrate **user-defined log quantities**.
* Explain the use of **Table** to display status information during a simulation run.
* Show that **Table** can write to a file.

## Boilerplate code

In [1]:
import hoomd
import datetime

In [2]:
import os
fn = os.path.join(os.getcwd(), 'log.txt')
![ -e "$fn" ] && rm "$fn"

## Define the Simulation

This tutorial executes the Lennard-Jones particle simulation from a previous tutorial. 
See [*Introducing Molecular Dyamics*](../01-Introducing-Molecular-Dynamics/00-index.ipynb) for a complete description of this code.

In [3]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu)
sim.create_state_from_gsd(filename='../01-Introducing-Molecular-Dynamics/random.gsd')

integrator = hoomd.md.Integrator(dt=0.005)
cell = hoomd.md.nlist.Cell()
lj = hoomd.md.pair.LJ(nlist=cell)
lj.params[('A', 'A')] = dict(epsilon=1, sigma=1)
lj.r_cut[('A', 'A')] = 2.5
integrator.forces.append(lj)
nvt = hoomd.md.methods.NVT(kT=1.5, filter=hoomd.filter.All(), tau=1.0)
integrator.methods.append(nvt)
sim.operations.integrator = integrator

## Formatted output

The **Table** writer formats log quantities in human-readable text and writes them to `stdout` or a file.
**Table** only supports `scalar` and `string` quantities due to the limitations of this format.
This section shows you how to use **Table** to display status information during a simulation run.

## Add quantities to a Logger

The `categories` argument to **Logger** defines the **categories** that it will accept.

In [4]:
logger = hoomd.logging.Logger(categories=['scalar', 'string'])

Log the and simulation `timestep` and `tps` quantities:

In [5]:
logger.add(sim, quantities=['timestep', 'tps'])

You can also log user-defined quantities using functions, callable class instances, or class properties.
For example, this class computes the estimated time remaining:

In [6]:
class Status():
    def __init__(self, sim):
        self.sim = sim
    
    @property
    def seconds_remaining(self):
        try:
            return (self.sim.final_timestep - self.sim.timestep) / self.sim.tps
        except ZeroDivisionError:
            return 0        
    
    @property
    def etr(self):
        return str(datetime.timedelta(seconds=self.seconds_remaining))


Assign the loggable quantity using the tuple `(object, property_name, flag)`,  where `flag` is the string name of the flag for this quantity. (The tuple for callable objects would be `(callable, flag)`).

In [7]:
status = Status(sim)
logger[('Status', 'etr')] = (status, 'etr', 'string')

Represent the namespace of your user-defined quantity with a tuple of strings - `('Status', 'etr'`) above.
You can use any number of arbitrary strings in the tuple to name your quantity.

## Display quantities with Table

**Table** is a **Writer** that formats the quantities in a **Logger** into a human readable table.
Create one that triggers periodically:

In [8]:
table = hoomd.write.Table(trigger=hoomd.trigger.Periodic(period=5000),
                          logger=logger)

Add it to the simulation:

In [9]:
sim.operations.writers.append(table)

Run the simulation and see the output:

In [10]:
sim.run(100000)

Simulation.timestep  Simulation.tps     Status.etr   
       15000           7541.26652     0:00:12.597353 
       20000           9597.93125     0:00:09.377021 
       25000          11375.52976     0:00:07.472179 
       30000          12542.47408     0:00:06.378327 
       35000          13355.34382     0:00:05.615730 
       40000          13965.07219     0:00:05.012505 
       45000          14433.20821     0:00:04.503503 
       50000          14803.43507     0:00:04.053113 
       55000          15099.66673     0:00:03.642464 
       60000          15355.09050     0:00:03.256249 
       65000          15568.67752     0:00:02.890419 
       70000          15754.07066     0:00:02.539026 
       75000          15907.18480     0:00:02.200264 
       80000          16042.17043     0:00:01.870071 
       85000          16166.14884     0:00:01.546441 
       90000          16271.30958     0:00:01.229157 
       95000          16366.43532     0:00:00.916510 
      100000          16449.

Later in this notebook, you are going to create another **Table Writer**.
Remove `table` now to avoid confusing the two:

In [11]:
sim.operations.writers.remove(table)

## Save Table output to a file

**Table** writes to `stdout` by default.
It can write to a file (or any Python file-like object with `write` and `flush` methods) instead.

Open the file to write:

In [12]:
file = open('log.txt', 'x')

Create a **Table** **Writer** that outputs to this file and add it to the **Simulation**:

In [13]:
table_file = hoomd.write.Table(output=file,
                               trigger=hoomd.trigger.Periodic(period=5000),
                               logger=logger)
sim.operations.writers.append(table_file)

Run the simulation:

In [14]:
sim.run(100000)

You can read the file with standard tools:

In [15]:
!tail log.txt

      165000          18050.84721     0:00:02.492958 
      170000          18049.81169     0:00:02.216090 
      175000          18058.62364     0:00:01.938132 
      180000          18068.19810     0:00:01.660376 
      185000          18074.87484     0:00:01.383135 
      190000          18074.76902     0:00:01.106515 
      195000          18081.95049     0:00:00.829557 
      200000          18086.99741     0:00:00.552883 
      205000          18092.71797     0:00:00.276354 
      210000          18092.99713        0:00:00     


In this section, you have displayed **loggable quantities** during a simulation run and saved them to a text file.
This is the last section in the logging tutorial.

[Previous section](03-Storing-Particle-Shape.ipynb) / [Tutorial index](../README.md)